# Data Leakage

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, make_scorer

## Learning Goals

- avoid letting information about test sets get into the training of models
- use best practices for building non-leaky workflows
- repair leaky workflows

## Introduction

We have encountered the idea of splitting our data into two, *training* our model on one bit and then *testing* it on the other.

The goal is to have an unbiased assessment of our model, and so we want to make sure that nothing about our test data sneaks into the training run of the model.

## A Mistake

Now consider the following workflow:

In [2]:
data = load_diabetes()

print(data.DESCR)

df = pd.concat([pd.DataFrame(data.data, columns=data.feature_names),
               pd.Series(data.target, name='target')], axis=1)

df.head()

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, T-Cells (a type of white blood cells)
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, thyroid stimulating hormone
      - s5      ltg, lamotrigine
      - s6      glu, blood sugar level

Note: Each of these 10 feature va

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [3]:
X, y = load_diabetes(return_X_y=True)

In [4]:
ss = StandardScaler().fit(X)
X_scld = ss.transform(X)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scld, y, random_state=42)

In [6]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print(lr.coef_, lr.intercept_)

[  2.27107279 -11.5103763   25.30316447  18.14921047 -43.68812386
  24.17505729   5.56228784  12.81809837  33.09612684   1.25207795] 151.66516982689885


<details>
    <summary>What did we do wrong</summary>
Well we fit the model only to our training data. Looks like we've done everything right, right?

It's important to understand that the answer here is a resounding "NO". It's true that we didn't directly fit our model to our test data. But the trouble is that we fit our scaler **to the whole dataset**. That is, records in the test data are contributing to calculations of column means and standard deviations, and so, surreptitiously, information about our test set is sneaking into the training run of the model after all!

To correct our mistake, we'll make sure to perform our train-test split **first**:
 
    
<code>X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=42)
    ss2 = StandardScaler().fit(X_train2)
    X_train2_scld = ss2.transform(X_train2)
    X_test2_scld = ss2.transform(X_test2)
    lr2 = LinearRegression()
    lr2.fit(X_train2_scld, y_train2)
    print(lr2.coef_, lr2.intercept_)
</code>
</details>

In [7]:
#correct code
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=42)

In [13]:
ss2 = StandardScaler().fit(X_train2)
X_train2_scld = ss2.transform(X_train2)
X_test2_scld = ss2.transform(X_test2)

lr2 = LinearRegression()
lr2.fit(X_train2_scld, y_train2)
print(lr2.coef_, lr2.intercept_)

[  2.21493322 -11.51452473  25.07685109  18.24943843 -44.14403151
  24.5135485    5.4971345   13.00640779  33.3797142    1.24791796] 154.34441087613294


Note that our model coefficients are slightly different from what they were before.

### Error Comparison

It's worth pointing out that, **for linear models**, there is **no** difference in modeling error:

In [14]:
y_test_hat = lr.predict(X_test)
mse = mean_squared_error(y_test, y_test_hat)
print(f"Our test RMSE for this model is {round(np.sqrt(mse), 2)}.")

Our test RMSE for this model is 53.37.


In [15]:
y_test2_hat = lr2.predict(X_test2_scld)
mse = mean_squared_error(y_test2, y_test2_hat)
print(f"Our test RMSE for this model is {round(np.sqrt(mse), 2)}.")

Our test RMSE for this model is 53.37.


This will **not** be true for other sorts of models that use different loss functions.

## Preprocessing

In general all preprocessing steps are subject to the same dangers here. Consider the preprocessing step of one-hot-encoding:

In [16]:
gun_poll = pd.read_csv('data/guns-polls.csv')

In [17]:
gun_poll.head()

,Question,Start,End,Pollster,Population,Support,Republican Support,Democratic Support,URL
0,age-21,2/20/18,2/23/18,CNN/SSRS,Registered Voters,72,61,86,http://cdn.cnn.com/cnn/2018/images/02/25/rel3a...
1,age-21,2/27/18,2/28/18,NPR/Ipsos,Adults,82,72,92,https://www.ipsos.com/en-us/npripsos-poll-majo...
2,age-21,3/1/18,3/4/18,Rasmussen,Adults,67,59,76,http://www.rasmussenreports.com/public_content...
3,age-21,2/22/18,2/26/18,Harris Interactive,Registered Voters,84,77,92,http://thehill.com/opinion/civil-rights/375993...
4,age-21,3/3/18,3/5/18,Quinnipiac,Registered Voters,78,63,93,https://poll.qu.edu/national/release-detail?Re...


In [18]:
gun_poll['Pollster'].value_counts()

YouGov                 12
Morning Consult        11
Quinnipiac              8
NPR/Ipsos               7
CNN/SSRS                5
CBS News                4
Rasmussen               2
Suffolk                 2
Harris Interactive      1
ABC/Washington Post     1
YouGov/Huffpost         1
SurveyMonkey            1
Harvard/Harris          1
Marist                  1
Name: Pollster, dtype: int64

Now if I were to fit a one-hot encoder to the whole `Pollster` column here, the encoder would learn all the categories. But I need to prepare myself for the real-world possibility that unfamiliar categories may show up in future records. Let's explore this.

In [19]:
# First I'll do a split
X_train, X_test = train_test_split(gun_poll, random_state=42)

Let's suppose now that I fit a `OneHotEncoder` to the `Pollster` column in my training data.

#### Exercise

Fit an encoder to the `Pollster` column of the training data and then check to see which categories are represented.

In [24]:
to_be_dummied = X_train[['Pollster']]
ohe = OneHotEncoder()
ohe.fit(to_be_dummied)
## So what categories do we have?
ohe.get_feature_names()

array(['x0_ABC/Washington Post', 'x0_CBS News', 'x0_CNN/SSRS',
       'x0_Marist', 'x0_Morning Consult', 'x0_NPR/Ipsos', 'x0_Quinnipiac',
       'x0_Rasmussen', 'x0_Suffolk', 'x0_YouGov', 'x0_YouGov/Huffpost'],
      dtype=object)

<details>
    <summary>Answer</summary>
<code>to_be_dummied = X_train[['Pollster']]
ohe = OneHotEncoder()
ohe.fit(to_be_dummied)
## So what categories do we have?
ohe.get_feature_names()</code>
</details>

We'll want to transform both train and test after we've fitted the encoder to the train.

In [25]:
test_to_be_dummied = X_test[['Pollster']]

ohe.transform(to_be_dummied)
ohe.transform(test_to_be_dummied)

ValueError: Found unknown categories ['SurveyMonkey', 'Harris Interactive', 'Harvard/Harris'] in column 0 during transform

There are categories in the testing data that don't appear in the training data! What should 
we do about that? (These values were not fit!)

### Approaches

- **Strategy 1**: Divide up the categories proportionally when we do our train_test_split. If we're using `sklearn`'s tool, that means taking advantage of the `stratify` parameter:

In [26]:
#Implement Strategy 1
train_test_split(gun_poll, stratify=gun_poll['Pollster'], random_state=42)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

Unfortunately, in this case, we can't use this since some categories have only a single member.

- **Strategy 2**: Drop the categories with very few representatives.

In the present case, let's try dropping the single-member categories.

In [30]:
#Implement Strategy 2
vc = gun_poll['Pollster'].value_counts()
vc_only_1 = vc[vc==1]
vc_only_1

Harris Interactive     1
ABC/Washington Post    1
YouGov/Huffpost        1
SurveyMonkey           1
Harvard/Harris         1
Marist                 1
Name: Pollster, dtype: int64

In [31]:
bad_cols = vc_only_1.index

In [32]:
bad_cols

Index(['Harris Interactive', 'ABC/Washington Post', 'YouGov/Huffpost',
       'SurveyMonkey', 'Harvard/Harris', 'Marist'],
      dtype='object')

In [34]:
gun_poll['Pollster'] = gun_poll['Pollster'].map(lambda x: np.nan if x in bad_cols else x)
gun_poll = gun_poll.dropna()


In [35]:
gun_poll['Pollster'].value_counts()

YouGov             12
Morning Consult    11
Quinnipiac          8
NPR/Ipsos           7
CNN/SSRS            5
CBS News            4
Suffolk             2
Rasmussen           2
Name: Pollster, dtype: int64

We could now split this carefully so that new categories don't show up in the testing data. In fact, now we can try the stratified split:

In [ ]:
#Perform a stratified split with those single-member categories dropped

In [39]:
X_train3, X_test3 = train_test_split(gun_poll, stratify=gun_poll['Pollster'], test_size = 0.3, random_state=42)

In [40]:
X_train3['Pollster'].value_counts()

YouGov             8
Morning Consult    8
Quinnipiac         6
NPR/Ipsos          5
CNN/SSRS           3
CBS News           3
Suffolk            1
Rasmussen          1
Name: Pollster, dtype: int64

In [41]:
X_test3['Pollster'].value_counts()

YouGov             4
Morning Consult    3
Quinnipiac         2
NPR/Ipsos          2
CNN/SSRS           2
Suffolk            1
Rasmussen          1
CBS News           1
Name: Pollster, dtype: int64

Now every category that appears in the test data appears also in the training data.

- **Strategy 3**: Adjust the settings on the one-hot-encoder.

For `sklearn`'s tool, we'll tweak the `handle_unknown` parameter:

#### Exercise

Fit a new encoder to our training data column that won't break when we try to use it to transform the test data. And then use the encoder to transform both train and test.

In [44]:
ohe2 = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe2.fit(to_be_dummied)

ohe2.transform(to_be_dummied)
ohe2.transform(test_to_be_dummied)

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

<details>
    <summary>Answer</summary>
<code>ohe2 = OneHotEncoder(handle_unknown='ignore')
ohe2.fit(to_be_dummied)
test_to_be_dummied = X_test[['Pollster']]
ohe2.transform(to_be_dummied)
ohe2.transform(test_to_be_dummied)</code>
</details>

## Leakage into Validation Data

If we employ cross-validation, then our training data points will be serving both for training and for validation. So there's a sense in which we can't help but let some information about our validation data sneak into the model.

But strictly speaking, cross-validation means building *multiple* models, and we still want each to be blind to its validation set.

The dangers of data leakage, therefore, are still very much real in the case of validation data. And they are often more subtle as well. Consider the following line of code:

In [45]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

In [46]:
# Using our scaled training data

cv_results = cross_validate(estimator=LinearRegression(),
                X=X_train2_scld,
                y=y_train2,
                scoring=mse,
                return_estimator=True)

In [47]:
# Looking at model coefficients on the first predictor

[model.coef_[0] for model in cv_results['estimator']]

[0.7680116279314555,
 -0.05303892863219852,
 2.5202065136668392,
 3.7246885179367135,
 4.554018949497536]

No leaks right?
We've built five models here, and none of them saw any points from the test data, so we have no leaks, right?

Wrong! We fit the `StandardScaler` to the whole training set, which means that information about *every* fold will affect every cross-validation. A better practice here would be to split our data into its cross-validation folds *first*. Then we can fit the scaler to only the training folds for each cross-validation.

Of course, the more preprocessing steps we have, the more tedious it becomes to do this work! For such tasks it is often greatly beneficial to take advantage of `sklearn`'s `Pipeline`s, which we'll have more to say about later.

For now, let's see if we can fix our leaky cross-validation scorer.



#### Exercise

Go back to `X_train2` and do the following:

- Split it into five validation folds (Use `KFold()`).
- For each split:
- (i) fit a `StandardScaler` to the four-fold chunk and transform all data points with it.
- (ii) fit a `LinearRegression` to the four-fold chunk and print out the value of the first coefficient.

In [ ]:
KFold().split(X_train2)

In [48]:
for train_ind, val_ind in KFold().split(X_train2):
    print(train_ind, val_ind)

[ 67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102
 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120
 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156
 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174
 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192
 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228
 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246
 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264
 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282
 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300
 301 302 303 304 305 306 307 308 309 310 311 312 31

In [52]:
for train_ind, val_ind in KFold().split(X_train2):
    train = X_train2[train_ind, :]
    val = X_train2[val_ind, : ]
    target_train = y_train2[train_ind]
    target_val = y_train2[val_ind]
    
    ss = StandardScaler().fit(train)
    train_scld = ss.transform(train)
    val_scld = ss.transform(val)
    
    lr = LinearRegression().fit(train_scld, target_train)
    
    print(lr.coef_[0])

0.7804849156410961
-0.052632372933910786
2.438099721585693
3.7975009826724104
4.560443723559545


<details>
    <summary>Answer</summary>

<code>for train_ind, val_ind in KFold().split(X_train2):
    train = X_train2[train_ind, :]
    val = X_train2[val_ind, :]
    target_train = y_train2[train_ind]
    target_val = y_train2[val_ind]
    ss = StandardScaler().fit(train)
    train_scld = ss.transform(train)
    val_scld = ss.transform(val)
    lr = LinearRegression().fit(train_scld, target_train)
    print(lr.coef_[0])</code>
</details>

### A Contrived But Illustrative Example

This is only a slight difference in coefficients, but for other datasets the difference can be great.

In [ ]:
fake_preds = np.array([6, 5, 2, 250, 300]).reshape(-1, 1)
fake_target = np.array([25, 30, 12, 400, 420]).reshape(-1, 1)

In [ ]:
df = pd.DataFrame(np.hstack((fake_preds, fake_target)),
                 columns=['pred', 'target'])

In [ ]:
df

In [ ]:
small_train = df[['pred']][:3]
small_val = df[['pred']][3:]
small_train_y = df['target'][:3]
small_val_y = df['target'][3:]

In [ ]:
# Scaling the whole dataset

ss = StandardScaler().fit(df[['pred']])

In [ ]:
X_tr = ss.transform(small_train)
X_va = ss.transform(small_val)

In [ ]:
lr = LinearRegression().fit(X_tr, small_train_y)
lr.coef_

In [ ]:
fig, ax = plt.subplots()

X = np.linspace(0, 310, 600)
y = lr.coef_ * X + lr.intercept_

ax.scatter(df['pred'], df['target'])
ax.plot(X, y)
ax.set_title(f"""The best-fit line when scaling the whole dataset is
    {round(lr.coef_[0])}X + {round(lr.intercept_)}""");

In [ ]:
# Splitting before scaling

ss2 = StandardScaler().fit(small_train)

In [ ]:
X_tr2 = ss2.transform(small_train)
X_va2 = ss2.transform(small_val)

In [ ]:
lr2 = LinearRegression().fit(X_tr2, small_train_y)
lr2.coef_

In [ ]:
fig, ax = plt.subplots()

X = np.linspace(0, 310, 600)
y = lr2.coef_ * X + lr2.intercept_

ax.scatter(df['pred'], df['target'])
ax.plot(X, y)
ax.set_title(f"""The best-fit line when scaling after splitting is
    {round(lr2.coef_[0])}X + {round(lr2.intercept_)}""");